In [ ]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import matplotlib as mpl
from pathlib import Path
from scipy.stats import mannwhitneyu

In [ ]:
import rpy2
# Configure for notebook use.
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(dplyr)
library(patchwork)

# Loading

In [ ]:
c0a = "accuracy (validation)"
c0b = "accuracy (test)"
c1 = "multiply-adds"

In [ ]:
# General 'knowledge'
improvement_direction = {
    "accuracy": 1,
    "accuracy (validation)": 1,
    "accuracy (test)": 1,
    "loss": -1,
    "loss (validation)": -1,
    "loss (test)": -1,
    "loss-clip": -1,
    "total bytes": -1,
    "total_memory_bytes": -1, # dict name
    "multiply-adds": -1,
    "total_mult_adds": -1, # dict name
    # "genotype": 0, # -- not a criterion
}
best_possible_value_imagenet = {
    "accuracy": 1.0,
    "loss": 0.0,
    "accuracy (validation)": 1.0,
    "loss (validation)": 0.0,
    "accuracy (test)": 1.0,
    "loss (test)": 0.0,
    "loss-clip": 0.0,
    "total bytes": 0.0,
    "total_memory_bytes": 0.0,
    "multiply-adds": 0.0,
    "total_mult_adds": 0.0, 
}

best_possible_value_voc = {
    "accuracy": 1.0,
    "loss": 0.0,
    "accuracy (validation)": 1.0,
    "loss (validation)": 0.0,
    "accuracy (test)": 1.0,
    "loss (test)": 0.0,
    "loss-clip": 0.0,
    "total bytes": 0.0,
    "total_memory_bytes": 0.0,
    "multiply-adds": 0.0,
    "total_mult_adds": 0.0, 
}

task_relabeling = {
    "voc": "VOC",
    "imagenet (a)": "ImageNet (a)",
    "imagenet (b)": "ImageNet (b)",
}

# # Task specific settings
# if task == "imagenet (a)":
#     # imagenet-a
#     folder = Path("./2024-01-02-results/imagenet_a/")
#     assert folder.exists()
#     run_folder = folder / "exp-imagenet-a"
#     assert run_folder.exists()
#     files = list(run_folder.glob("*.arrow"))
#     reference_file = folder / "stitched-imagenet-a-reference.arrow"
#     # some task-specific tidbits
#     min_accuracy = 0.7
#     best_possible_value["accuracy (validation)"] = 0.80
#     best_possible_value["accuracy (test)"] = 0.85
# elif task == "segmentation":
#     folder = Path("./2024-01-02-results/segmentation/")
#     assert folder.exists()
#     run_folder = folder / "exp-voc"
#     assert run_folder.exists()
#     files = list(run_folder.glob("*.arrow"))
#     reference_file = folder / "stitched-voc-reference.arrow"
#     # some task-specific tidbits
#     min_accuracy = 0.90
# else:
#     raise ValueError("Unknown task")
# if c0 == "accuracy":
#     min_performance = min_accuracy

In [ ]:
datapoints = (pl.concat([
    pl.read_ndjson("./2024-01-02-results/imagenet_a/imagenet_a-front.reeval.jsonl").lazy().with_columns(pl.lit("imagenet (a)").alias("task")),
    pl.read_ndjson("./2024-01-02-results/segmentation/segmentation-front.reeval.jsonl").lazy().with_columns(pl.lit("voc").alias("task")),
    pl.read_ndjson("./2024-01-04/imagenet_b-front.reeval.jsonl").lazy().with_columns(pl.lit("imagenet (b)").alias("task")),
], how="diagonal")
    .with_columns([
        pl.col("set").str.replace("_", "-"),
        pl.col("accuracy").alias("accuracy (validation)"),
        pl.col("reeval-result").struct.field("accuracy").alias("accuracy (test)"),
        ])
    .drop("reeval-result")
    .with_columns([
        pl.col("set").replace({"SGA": "GA"}),
        pl.col("task").replace(task_relabeling),
        ])
).collect()
datapoints

> Random Search had a bug that caused it to reevaluate the same solution over and over again from a certain point onwards.
> **Exclude** these results.

In [ ]:
datapoints = datapoints.filter(pl.col("set") != "RS")

# Prepare for front computation

In [ ]:
# Tools for computing fronts
def maybe_over(a, o):
    if len(o) == 0: return a
    else: return a.over(o)

def compute_pareto(df, group_vars, c0, c1, dmul=1):
    return (df.sort([c0, c1], descending=[improvement_direction[c0] * dmul > 0, improvement_direction[c1] * dmul > 0])
        .with_columns((pl.col(c1) * -improvement_direction[c1] * dmul).alias("c1-min"))
        .with_columns(maybe_over((pl.col("c1-min")).cum_min(), group_vars).alias("mv"))
        .with_columns((maybe_over(pl.col("c1-min") < pl.col("mv").shift(1), group_vars).alias("is pareto")).fill_null(True))
        .filter(pl.col("is pareto"))
    )

def compute_2d_hv(df_pareto, ref, axis_scale, group_vars, c0, c1):
    # note - df_pareto is a df created using compute_pareto
    dhva = (df_pareto.sort(c0, descending=improvement_direction[c0] < 0)
        # Samples worse than reference point do not contribute.
        # .filter(improvement_direction[c0] * pl.col(c0) > improvement_direction[c0] * ref[0])
        # .filter(improvement_direction[c1] * pl.col(c1) > improvement_direction[c1] * ref[1])
        .with_columns(
        [
            maybe_over( improvement_direction[c0] * (pl.col(c0) - pl.col(c0).shift(1).fill_null(ref[0])) / axis_scale[0], group_vars).alias("slice_width"),
            maybe_over( improvement_direction[c1] * (pl.col(c1) - ref[1]) / axis_scale[1], group_vars).alias("slice_height"),
        ])
        .select([pl.col(group_vars), (pl.col("slice_width") * pl.col("slice_height")).alias("hv_contrib")])
        .group_by(group_vars).agg(pl.col("hv_contrib").sum()))
    return dhva

def get_transformed_front(per_run_front, grouping, c0, c1, include_poly=False):
    # Create a dataframe for plotting in R
    per_run_front_b = (per_run_front.lazy()
        # Add a tag so that we can track which samples were original - and which ones were added for sake
        # of continuing the lines.
        .with_columns(pl.lit(1.0).alias("is_original"))
        # For each run include an additional two rows:
        # Repeat best per objective, but replace the other objective with -Inf - as to plot towards the axes.
        .merge_sorted(per_run_front.lazy()
                    .with_columns([(pl.col(c0) * improvement_direction[c0]).alias("c0-n"),
                                    pl.lit(-np.Inf * improvement_direction[c1]).alias(c1),
                                    pl.lit(0.0).alias("is_original")])
                    .group_by(grouping, maintain_order=True)
                    .agg(pl.all().sort_by("c0-n").last())
                    .select(per_run_front.columns + ["is_original"]), "file")
        .merge_sorted(per_run_front.lazy()
                    .with_columns([(pl.col(c1) * improvement_direction[c1]).alias("c1-n"),
                                    pl.lit(-np.Inf * improvement_direction[c0]).alias(c0),
                                    pl.lit(0.0).alias("is_original")])
                    .group_by(grouping, maintain_order=True)
                    .agg(pl.all().sort_by("c1-n").last())
                    .select(per_run_front.columns + ["is_original"]), "file")
        # Add c0 and c1 as a named column
        .with_columns([pl.col(c0).alias("c0"), pl.col(c1).alias("c1")])
        # Sort, for good measure
        .sort(grouping + [c0, c1]).collect())
    if include_poly:
        # Also derive a polygon-boundary
        poly_b = pl.concat([
            per_run_front_b.lazy().select(
                pl.col(grouping),
                pl.col("c0"),
                pl.col("c1"),
            ).with_row_count().with_columns(pl.col("row_nr") * 2 + 1),
            per_run_front_b.lazy().select(
                pl.col(grouping),
                pl.col("c0").shift(1).fill_null(-np.Inf * improvement_direction[c0]).over(grouping),
                pl.col("c1"),
            ).with_row_count().with_columns(pl.col("row_nr") * 2),
            per_run_front_b.lazy().group_by(grouping).agg(
                pl.lit(-np.Inf * improvement_direction[c0]).alias("c0"),
                pl.lit(-np.Inf * improvement_direction[c1]).alias("c1"),
                pl.lit(-1).alias("row_nr"),
            )
        ], how="diagonal_relaxed").sort("row_nr").collect()
        return per_run_front_b.to_pandas(), poly_b.to_pandas()

    return per_run_front_b.to_pandas()

def compute_reference_domination_poly(reference_points, grouping, c0, c1):
    return (pl.concat([
            reference_points.lazy().select(
                pl.col(grouping),
                pl.col("c0"),
                pl.col("c1"),
            ).with_row_count().with_columns(pl.col("row_nr") * 2),
            reference_points.lazy().select(
                pl.col(grouping),
                pl.col("c0").shift(-1).fill_null(np.Inf * improvement_direction[c0]).over(grouping),
                pl.col("c1"),
            ).with_row_count().with_columns(pl.col("row_nr") * 2 + 1),
            reference_points.lazy().group_by(grouping).agg(
                (pl.col("c0") * improvement_direction[c0]).min(),
                pl.lit(np.Inf * improvement_direction[c1]).alias("c1"),
                pl.lit(-1).alias("row_nr"),
            ),
            reference_points.lazy().group_by(grouping).agg(
                pl.lit(np.Inf * improvement_direction[c0]).alias("c0"),
                pl.lit(np.Inf * improvement_direction[c1]).alias("c1"),
                pl.lit(-2).alias("row_nr"),
            ),
        ], how="diagonal_relaxed").sort(grouping + ["row_nr"])).collect()


# Compute front over all evaluated solutions

In [ ]:
# Original
all_data = datapoints
total_front_a = compute_pareto(all_data.lazy().sort(["task", c0a, c1]), ["task"], c0a, c1).drop("genotype").collect()
reference_a = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0a).alias("c0"), pl.col(c1).alias("c1")
)
total_front_b = compute_pareto(all_data.lazy().sort(["task", c0b, c1]), ["task"], c0b, c1).collect()
reference_b = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0b).alias("c0"), pl.col(c1).alias("c1")
)

In [ ]:
pd_per_run_front = get_transformed_front(total_front_a, ["task"], c0a, c1)
reference = reference_a.to_pandas()
reference_poly = compute_reference_domination_poly(compute_pareto(reference_a, ["task"], c0a, c1, dmul=-1), ["task"], c0a, c1).to_pandas()
c0 = c0a

In [ ]:
reference_poly

In [ ]:
# total_front_a
pd_per_run_front

In [ ]:
%%R -i pd_per_run_front -i reference_poly -i reference -i c0 -i c1 -w 500 -h 300

# Remove additional samples added to continue the lines to the axis edge
pd_per_run_front_excl_edges <- pd_per_run_front |> filter(`is_original` > 0.5)

pla <- ggplot(pd_per_run_front, aes(x = `c0`, y=`c1`)) +
    geom_polygon(data = reference_poly, fill="orange", alpha=0.2) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_per_run_front_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", group="reference", shape=4, size=2, stroke=2) +
    labs(x = c0, y = c1, color = "approach") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
pla

In [ ]:
pd_per_run_front, poly = get_transformed_front(total_front_b.sort(["task", c0b, c1]), ["task"], c0b, c1, include_poly=True)
reference_poly = compute_reference_domination_poly(compute_pareto(reference_b, ["task"], c0b, c1, dmul=-1), ["task"], c0b, c1).to_pandas()
reference = reference_b.to_pandas()
c0 = c0b

In [ ]:
%%R -i pd_per_run_front -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300

# Remove additional samples added to continue the lines to the axis edge
pd_per_run_front_excl_edges <- pd_per_run_front |> filter(`is_original` > 0.5)

plb <- ggplot(pd_per_run_front, aes(x = `c0`, y=`c1`)) +
    geom_polygon(data = reference_poly, fill="orange", alpha=0.2) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_per_run_front_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", group="reference", shape=4, size=2, stroke=2) +
    labs(x = c0, y = c1, color = "approach") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
plb

In [ ]:
%%R -w 1000 -h 300
pla | plb

# Compute front per run
With respect to a reference depending on all evaluated solutions / predetermined bounds.

In [ ]:
# # Compute reference point & scale for hypervolume
# # For the reference point we use the nadir: the worst combination of objectives.
# # Note, that this is computed /after/ a minimal fitness filter.
# worst_perf_c0, worst_perf_c1 = all_data.select(
#     (improvement_direction[c0] * (improvement_direction[c0] * pl.col(c0)).min()).alias(c0),
#     (improvement_direction[c1] * (improvement_direction[c1] * pl.col(c1)).min()).alias(c1)
#     ).collect().row(0)
# # Override, if we have a filter threshold.
# if min_performance is not None:
#     worst_perf_c0 = min_performance
# scale_c0 = -improvement_direction[c0] * (worst_perf_c0 - best_possible_value[c0])
# scale_c1 = -improvement_direction[c1] * (worst_perf_c1 - best_possible_value[c1])

# hv_reference_point = (worst_perf_c0, worst_perf_c1)
# hv_scale = (scale_c0, scale_c1)

# print(f"Reference point: ({worst_perf_c0}, {worst_perf_c1}) - Scale: ({scale_c0}, {scale_c1})")

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy().sort(["task", "file", c0a, c1]), ["task", "file"], c0a, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["task", "file"], c0a, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0a).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0a, c1, dmul=-1), ["task"], c0a, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0a

In [ ]:
scale_multiplier = pl.from_pandas(pd_fronts).lazy().select(pl.col("c1").min().log10().floor()).collect()
scale_multiplier

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300
pal <- palette("r4")[2:6]
scientific_10 <- function(x) {   parse(text=gsub("e\\+*", " %*% 10^", scales::scientific_format()(x))) }
# scientific_10 <- function(x) { x / 10^9 } 

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

pla <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = c1, color = "approach: ") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
pla

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy(), ["task", "file"], c0b, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["task", "file"], c0b, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0b).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0b, c1, dmul=-1), ["task"], c0b, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0b

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

plb <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = c1, color = "approach: ") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
plb

In [ ]:
%%R -w 1000 -h 260
pl <- (pla | plb) + plot_layout(guides = "collect") & theme(legend.position="bottom")
# ggsave("fronts-in-ab-seg-dominates-parent-marker.pdf", device = cairo_pdf)
# ggsave("fronts-in-a-seg-dominates-parent-marker.pdf", device = cairo_pdf)
pl

### b solo

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy().sort(["file", c0a, c1]), ["file"], c0a, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["file"], c0a, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0a).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0a, c1, dmul=-1), ["task"], c0a, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0a

In [ ]:
scale_multiplier = pl.from_pandas(pd_fronts).lazy().select(pl.col("c1").min().log10().floor()).collect()
scale_multiplier

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300
pal <- palette("r4")[c(2,4)]
scientific_10 <- function(x) {   parse(text=gsub("e\\+*", " %*% 10^", scales::scientific_format()(x))) }
# scientific_10 <- function(x) { x / 10^9 } 

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

pla <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = c1, color = "approach: ") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
pla

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy(), ["file"], c0b, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["file"], c0b, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0b).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0b, c1, dmul=-1), ["task"], c0b, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0b

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

plb <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = c1, color = "approach: ") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
plb

In [ ]:
%%R -w 500 -h 260
pl <- (pla | plb) + plot_layout(guides = "collect") & theme(legend.position="bottom")
# ggsave("fronts-in-ab-seg-dominates-parent-marker.pdf")
ggsave("fronts-in-b-dominates-parent-marker.pdf", device = cairo_pdf)
pl

### Rescaled

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy().sort(["task", "file", c0a, c1]), ["task", "file"], c0a, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["task", "file"], c0a, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0a).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0a, c1, dmul=-1), ["task"], c0a, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0a

In [ ]:
scale_multiplier = pl.from_pandas(pd_fronts).lazy().select(pl.col("c1").min().log10().floor()).collect()
scale_multiplier

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300
pal <- palette("r4")[2:6]
# scientific_10 <- function(x) {   parse(text=gsub("e\\+*", " %*% 10^", scales::scientific_format()(x))) }
scientific_10 <- function(x) { x / 10^9 } 

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

pla <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = bquote(.(c1) ~~ (phantom("")%*% 10^9)), color = "approach") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
pla

In [ ]:
all_data = datapoints
fronts_a = compute_pareto(all_data.lazy(), ["file"], c0b, c1).drop("genotype").collect()
pd_fronts = get_transformed_front(fronts_a, ["file"], c0b, c1)
reference = all_data.filter(pl.col("file") == "reference").drop("genotype").with_columns(
    pl.col(c0b).alias("c0"), pl.col(c1).alias("c1")
)
reference_poly = compute_reference_domination_poly(compute_pareto(reference, ["task"], c0b, c1, dmul=-1), ["task"], c0b, c1).to_pandas()
reference = reference.to_pandas()
c0 = c0b

In [ ]:
%%R -i pd_fronts -i reference -i reference_poly -i c0 -i c1 -w 500 -h 300

# Remove additional samples added to continue the lines to the axis edge
pd_fronts_no_ref <- pd_fronts |> filter(`set` != "reference")
pd_fronts_excl_edges <- pd_fronts_no_ref |> filter(`is_original` > 0.5)

plb <- ggplot(pd_fronts_no_ref, aes(x = `c0`, y=`c1`, color=`set`, group=`file`)) +
    geom_polygon(data = reference_poly, aes(x=`c0`, y=`c1`), fill="orange", alpha=0.2, inherit.aes=FALSE) +
    geom_step(alpha=0.3, direction = "vh") +
    geom_point(data = pd_fronts_excl_edges, alpha=0.3) +
    geom_point(data = reference, color="orange", shape=4, size=2, stroke=2) +
    scale_color_manual(values=pal) +
    scale_y_continuous(label=scientific_10) +
    labs(x = c0, y = bquote(.(c1) ~~ (phantom("")%*% 10^9)), color = "approach") +
    facet_wrap(`task` ~ ., scales="free") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())
plb

In [ ]:
%%R -w 1000 -h 260
pl <- (pla | plb) + plot_layout(guides = "collect") & theme(legend.position="bottom")
ggsave("fronts-in-ab-seg-dominates-parent-marker-rs.pdf")
pl

## Hypervolume

In [ ]:
frac_offset = 0.01

def compute_reference_point_and_norm(df, c0, c1):
    return (df.lazy()
    # Obtain min / max per task
    .group_by("task")
    .agg([
        ((pl.col(c0) * improvement_direction[c0]).min() * improvement_direction[c0]).alias("c0-worst"),
        ((pl.col(c0) * improvement_direction[c0]).max() * improvement_direction[c0]).alias("c0-best"),
        ((pl.col(c1) * improvement_direction[c1]).min() * improvement_direction[c1]).alias("c1-worst"),
        ((pl.col(c1) * improvement_direction[c1]).max() * improvement_direction[c1]).alias("c1-best"),
    ])
    # Obtain ranges
    .with_columns([
        (pl.col("c0-best") - pl.col("c0-worst")).abs().alias("c0-range"),
        (pl.col("c1-best") - pl.col("c1-worst")).abs().alias("c1-range"),
    ])
    # Determine reference point and normalization factor
    .select(
        pl.col("task"),
        (pl.col("c0-worst") - (pl.col("c0-range") * (frac_offset / 2))).alias("c0-ref"),
        (pl.col("c1-worst") - (pl.col("c1-range") * (frac_offset / 2))).alias("c1-ref"),
        (pl.col("c0-range") * (1.0 + frac_offset)).alias("c0-norm"),
        (pl.col("c1-range") * (1.0 + frac_offset)).alias("c1-norm"),
    )
    )

def compute_2d_hv_refdf(df_pareto, ref_df, group_vars, c0, c1):
    # note - df_pareto is a df created using compute_pareto
    dhva = (df_pareto.lazy()
            .join(ref_df, on="task")
            .sort(c0, descending=improvement_direction[c0] < 0)
        # Samples worse than reference point do not contribute.
        # .filter(improvement_direction[c0] * pl.col(c0) > improvement_direction[c0] * ref[0])
        # .filter(improvement_direction[c1] * pl.col(c1) > improvement_direction[c1] * ref[1])
        .with_columns(
        [
            maybe_over( improvement_direction[c0] * (pl.col(c0) - pl.col(c0).shift(1).fill_null(pl.col("c0-ref"))) / pl.col("c0-norm"), group_vars).alias("slice_width"),
            maybe_over( improvement_direction[c1] * (pl.col(c1) - pl.col("c1-ref")) / pl.col("c1-norm"), group_vars).alias("slice_height"),
        ])
        .select([pl.col(group_vars), (pl.col("slice_width") * pl.col("slice_height")).alias("hv_contrib")])
        .group_by(group_vars).agg(pl.col("hv_contrib").sum())
        )
    return dhva

## Validation

In [ ]:
metrics = compute_reference_point_and_norm(datapoints, c0a, c1)
hypervolumes = compute_2d_hv_refdf(datapoints, metrics, ["task", "set", "file"], c0a, c1)

metrics, hypervolumes = pl.collect_all((metrics, hypervolumes))

In [ ]:
metrics

In [ ]:
from IPython.display import display
with pl.Config(tbl_cols=40, tbl_rows=42) as cfg:
    display(hypervolumes.sort(["task", "hv_contrib"]))

In [ ]:
p_threshold = 0.05
perform_correction = True

c_hv_mean = "mean"
c_hv_std = "std"
c_hv_q10 = "q10"
c_hv_median = "median"
c_hv_q90 = "q90"


column_ordering = {c_hv_mean: 0, c_hv_std: 1, c_hv_q10: 2, c_hv_median: 3, c_hv_q90: 4 }
def sort_key(a):
    return [column_ordering.get(r, r) for r in a]

hvl = (hypervolumes.lazy()
    .rename({"set": "approach"})
    .group_by(["task", "approach"])
    .agg([
        pl.col("hv_contrib"),
        # pl.col("hv_contrib").mean().alias(c_hv_mean),
        # pl.col("hv_contrib").std().alias(c_hv_std),
        pl.col("hv_contrib").quantile(0.1).alias(c_hv_q10),
        pl.col("hv_contrib").median().alias(c_hv_median),
        pl.col("hv_contrib").quantile(0.9).alias(c_hv_q90),
    ]))

# 
best = (hvl
        .sort(pl.col("task", c_hv_median), descending=True)
        .group_by("task")
        .first())
best.collect()

paired = (hvl
          .join(best, on="task")
          .with_columns(
              pl.struct(["hv_contrib", "hv_contrib_right"]).map_elements(
              lambda c: mannwhitneyu(c["hv_contrib"], c["hv_contrib_right"], alternative="less")._asdict()
              ,return_dtype=pl.Struct({"statistic": pl.Float64, "pvalue": pl.Float64})).alias("mwu"))
          .select(pl.all().exclude("^.*_right$").exclude("mwu"),
                  pl.col("mwu").struct.field("statistic").alias("statistic"),
                  pl.col("mwu").struct.field("pvalue").alias("pvalue"),
                  (pl.col("approach") == pl.col("approach_right")).alias("is_best"),
                  # Don't test against self & reference - testing against self is useless & testing against reference is moot:
                  # reference is a distribution that contains only one value - which is extremely annoying to reason about...
                  (~((pl.col("approach") == pl.col("approach_right")) | (pl.col("approach") == "reference"))).alias("include_in_p_test"),
                )
          .sort(["task", "pvalue"])
          .with_columns(
              (pl.col("include_in_p_test").sum().over("task") - pl.col("include_in_p_test").cum_sum().over("task") + 1).alias("c")
          )
          .with_columns(
              (pl.lit(p_threshold) / (1.0 if not perform_correction else pl.col("c"))).alias("p_threshold")
            )
          .with_columns(
              ((pl.col("pvalue") < pl.col("p_threshold"))).cast(int).cum_min().cast(bool).over("task").alias("null reject")
          )
          .with_columns(
              (~pl.col("null reject") & (pl.col("include_in_p_test") | pl.col("is_best"))).alias("near_best")
                  )
)
# .select(pl.col(["approach", "task", c_hv_q10, c_hv_median, c_hv_q90, "near_best"]))
hvl = paired.collect()
# incl mean and std: [c_hv_mean, c_hv_std, c_hv_q10, c_hv_median, c_hv_q90]

def bold_entry(val):
    # Query
    near_best = (pl.DataFrame([{"task": val.name[0], "approach": list(val.index)}])
         .lazy()
         .explode("approach")
         .join(hvl.lazy(), on=["task", "approach"], how="left")
         .select(pl.col("near_best").fill_null(False))
        ).collect()
    return ["font-weight: bold" if is_near_best else "" for is_near_best in near_best["near_best"]]


# Pandas has a few more tools for creating tables...
hv_table = (hvl
    .to_pandas()
    .pivot(index=["approach"], columns=["task"], values=[c_hv_q10, c_hv_median, c_hv_q90])
    .reorder_levels([1, 0], axis=1)
    .sort_index(axis=1, key=sort_key)
    .style
    .format(na_rep="-", precision=4)
    .apply(bold_entry)
)
hv_table.to_excel("./2024-01-15-hypervolume-table-validation-sig.xlsx")
hv_table

In [ ]:
hvl

In [ ]:
(hvl.lazy()
    .select(pl.all().exclude(["hv_contrib", "null reject", "p_threshold", "c"]))
    .sort(["task", "approach"])
).collect().to_pandas().to_excel("2024-01-15-hypervolume-table-validation-sig-detailed-supplement.xlsx")

## Test

In [ ]:
metrics = compute_reference_point_and_norm(datapoints, c0b, c1)
hypervolumes = compute_2d_hv_refdf(compute_pareto(datapoints, ["task", "set", "file"], c0b, c1), metrics, ["task", "set", "file"], c0b, c1)

metrics, hypervolumes = pl.collect_all((metrics, hypervolumes))

In [ ]:
metrics_tab = metrics.to_pandas().style
metrics_tab.to_excel("2024-01-15-hypervolume-table-test-reference.xlsx")
metrics_tab

In [ ]:
from IPython.display import display
with pl.Config(tbl_cols=40, tbl_rows=42) as cfg:
    display(hypervolumes.sort(["task", "hv_contrib"]))

In [ ]:
p_threshold = 0.05
perform_correction = True

c_hv_mean = "mean"
c_hv_std = "std"
c_hv_q10 = "q10"
c_hv_median = "median"
c_hv_q90 = "q90"


column_ordering = {c_hv_mean: 0, c_hv_std: 1, c_hv_q10: 2, c_hv_median: 3, c_hv_q90: 4 }
def sort_key(a):
    return [column_ordering.get(r, r) for r in a]

hvl = (hypervolumes.lazy()
    .rename({"set": "approach"})
    .group_by(["task", "approach"])
    .agg([
        pl.col("hv_contrib"),
        # pl.col("hv_contrib").mean().alias(c_hv_mean),
        # pl.col("hv_contrib").std().alias(c_hv_std),
        pl.col("hv_contrib").quantile(0.1).alias(c_hv_q10),
        pl.col("hv_contrib").median().alias(c_hv_median),
        pl.col("hv_contrib").quantile(0.9).alias(c_hv_q90),
    ]))

# 
best = (hvl
        .sort(pl.col("task", c_hv_median), descending=True)
        .group_by("task")
        .first())
best.collect()

paired = (hvl
          .join(best, on="task")
          .with_columns(
              pl.struct(["hv_contrib", "hv_contrib_right"]).map_elements(
              lambda c: mannwhitneyu(c["hv_contrib"], c["hv_contrib_right"], alternative="less")._asdict()
              ,return_dtype=pl.Struct({"statistic": pl.Float64, "pvalue": pl.Float64})).alias("mwu"))
          .select(pl.all().exclude("^.*_right$").exclude("mwu"),
                  pl.col("mwu").struct.field("statistic").alias("statistic"),
                  pl.col("mwu").struct.field("pvalue").alias("pvalue"),
                  (pl.col("approach") == pl.col("approach_right")).alias("is_best"),
                  # Don't test against self & reference - testing against self is useless & testing against reference is moot:
                  # reference is a distribution that contains only one value - which is extremely annoying to reason about...
                  (~((pl.col("approach") == pl.col("approach_right")) | (pl.col("approach") == "reference"))).alias("include_in_p_test"),
                )
          .sort(["task", "pvalue"])
          .with_columns(
              (pl.col("include_in_p_test").sum().over("task") - pl.col("include_in_p_test").cum_sum().over("task") + 1).alias("c")
          )
          .with_columns(
              (pl.lit(p_threshold) / (1.0 if not perform_correction else pl.col("c"))).alias("p_threshold")
            )
          .with_columns(
              ((pl.col("pvalue") < pl.col("p_threshold"))).cast(int).cum_min().cast(bool).over("task").alias("null reject")
          )
          .with_columns(
              (~pl.col("null reject") & (pl.col("include_in_p_test") | pl.col("is_best"))).alias("near_best")
                  )
)
# .select(pl.col(["approach", "task", c_hv_q10, c_hv_median, c_hv_q90, "near_best"]))
hvl = paired.collect()
# incl mean and std: [c_hv_mean, c_hv_std, c_hv_q10, c_hv_median, c_hv_q90]

def bold_entry(val):
    # Query
    near_best = (pl.DataFrame([{"task": val.name[0], "approach": list(val.index)}])
         .lazy()
         .explode("approach")
         .join(hvl.lazy(), on=["task", "approach"], how="left")
         .select(pl.col("near_best").fill_null(False))
        ).collect()
    return ["font-weight: bold" if is_near_best else "" for is_near_best in near_best["near_best"]]


# Pandas has a few more tools for creating tables...
hv_table = (hvl
    .to_pandas()
    .pivot(index=["approach"], columns=["task"], values=[c_hv_q10, c_hv_median, c_hv_q90])
    .reorder_levels([1, 0], axis=1)
    .sort_index(axis=1, key=sort_key)
    .style
    .format(na_rep="-", precision=4)
    .apply(bold_entry)
)
hv_table.to_excel("./2024-01-15-hypervolume-table-test-sig.xlsx")
hv_table

In [ ]:
hvl

In [ ]:
(hvl.lazy()
    .select(pl.all().exclude(["hv_contrib", "null reject", "p_threshold", "c"]))
    .sort(["task", "approach"])
).collect().to_pandas().to_excel("2024-01-15-hypervolume-table-test-sig-detailed-supplement.xlsx")